In [8]:
import os

while os.getcwd().split("/")[-1] != "alfa-hack-rag":
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [9]:
import pandas as pd

In [ ]:
reference_df = pd.read_csv("ranking_results/32_submit_FRIDA.csv")
test_df = pd.read_csv("ranking_results/submit_ru_en_RoSBERTa.csv")

In [11]:
def parse_web_list(web_list_series):
    return web_list_series.str[1:-1].str.split(", ").apply(lambda x: [int(num) for num in x])

In [12]:
reference_df["web_list"] = parse_web_list(reference_df["web_list"])
test_df["web_list"] = parse_web_list(test_df["web_list"])

In [13]:
def calculate_hit_at_k(reference_df, test_df, k=5):
    """
    Рассчитывает метрику Hit@k и возвращает DataFrame с детальными результатами

    Parameters:
    reference_df: DataFrame с эталонными данными
    test_df: DataFrame с предсказанными данными
    k: количество top результатов для проверки

    Returns:
    tuple: (hit_at_k, results_df) где results_df содержит детальную информацию по каждому вопросу
    """
    
    results = []
    valid_questions = 0
    total_hits = 0

    # Проходим по всем вопросам в тестовом DataFrame
    for _, test_row in test_df.iterrows():
        q_id = test_row["q_id"]
        test_webs = test_row["web_list"][:k]  # Берем первые k элементов

        # Находим соответствующий вопрос в эталонном DataFrame
        reference_row = reference_df[reference_df["q_id"] == q_id]

        # Если вопрос есть в эталонных данных
        if not reference_row.empty:
            reference_webs = reference_row.iloc[0]["web_list"]

            # Проверяем, есть ли хотя бы одно пересечение
            intersection = set(test_webs) & set(reference_webs)
            hit = 1 if intersection else 0
            
            total_hits += hit
            valid_questions += 1

            # Сохраняем детальную информацию
            results.append({
                'q_id': q_id,
                'reference_webs': reference_webs,
                'test_webs': test_webs,
                'intersection': list(intersection),
                'intersection_count': len(intersection),
                'hit@k': hit
            })

    # Создаем DataFrame с результатами
    results_df = pd.DataFrame(results)
    
    # Рассчитываем итоговую метрику
    if valid_questions > 0:
        hit_at_k = total_hits / valid_questions
    else:
        hit_at_k = 0

    return hit_at_k, results_df

# Пример использования:
hit_at_5, results_df = calculate_hit_at_k(reference_df, test_df, k=5)
print(f"Hit@5 = {hit_at_5:.4f}")
results_df

Hit@5 = 0.7146


,q_id,reference_webs,test_webs,intersection,intersection_count,hit@k
0,1,"[1157, 372, 1896, 593, 1098]","[1567, 372, 135, 92, 1825]",[372],1,1
1,2,"[1157, 1098, 1080, 135, 368]","[368, 372, 1607, 1555, 1045]",[368],1,1
2,3,"[1900, 116, 114, 1902, 110]","[1915, 516, 692, 677, 1934]",[],0,0
3,4,"[796, 1043, 1006, 1900, 1902]","[1915, 408, 1592, 1043, 1924]",[1043],1,1
4,5,"[1193, 1029, 1034, 1025, 156]","[1586, 1193, 1033, 1029, 1040]","[1193, 1029]",2,1
...,...,...,...,...,...,...
6972,6973,"[1902, 420, 716, 1197, 1516]","[420, 1902, 1516, 1197, 1552]","[420, 1197, 1902, 1516]",4,1
6973,6974,"[1905, 1627, 66, 1527, 86]","[730, 1905, 731, 626, 911]",[1905],1,1
6974,6975,"[886, 1042, 1193, 796, 1607]","[1592, 1043, 1590, 1608, 211]",[],0,0
6975,6976,"[1567, 16, 1072, 1896, 1125]","[1072, 123, 1567, 37, 92]","[1072, 1567]",2,1
